In [8]:
import ROOT, glob, os
from array import array
ROOT.gROOT.SetBatch(True)
ROOT.TH1.SetDefaultSumw2(True)

#=====uproot=======
files = sorted(glob.glob("output_uproot/hist_20250822_130953_Run3Summer22NanoAODv12_*.root"))[:32]

#======root========
#files = sorted(glob.glob("output/hist_20250822_130953_Run3Summer22NanoAODv12_*.root"))[:32]

pT_bins = [200, 250, 300, 350, 400, 500, 600, 700, 800]
pt_edges = array('d', pT_bins)

h_msd      = ROOT.TH1F("h_msd", "Msoftdrop;M_{softdrop};Events", 40, 0, 200)
h_pt       = ROOT.TH1F("h_pt",       "FatJet p_{T};p_{T} [GeV];Events", 40, 0, 1000)
h_dr       = ROOT.TH1F("h_dr",       "#DeltaR(FatJet, Z);#DeltaR;Events", 40, 0, 2.0)
h_den_msd  = ROOT.TH1F("h_den_msd",  "Denominator; msoftdrop [GeV];Events", 40, 0, 200)
h_num_msd  = ROOT.TH1F("h_num_msd",  "Numerator;   msoftdrop [GeV];Events", 40, 0, 200)
h_den_dr   = ROOT.TH1F("h_den_dr",   "All pairs;#DeltaR(FatJet,Z);Events", 40, 0.0, 2.0)
h_num_dr   = ROOT.TH1F("h_num_dr",   "Matched (pass #DeltaR);#DeltaR(FatJet,Z);Events", 40, 0.0, 2.0)
h_den_pt   = ROOT.TH1F("h_den_pt",   "Denominator;p_{T} [GeV];Events", len(pT_bins)-1, pt_edges)
h_num_pt   = ROOT.TH1F("h_num_pt",   "Numerator;p_{T} [GeV];Events",   len(pT_bins)-1, pt_edges)

for h in [h_msd, h_pt, h_dr, h_den_msd, h_num_msd, h_den_dr, h_num_dr, h_den_pt, h_num_pt]:
    h.SetDirectory(0)
    
for f in files:
    tf = ROOT.TFile.Open(f)
    for name, h in [("h_msd", h_msd), ("h_pt", h_pt), ("h_dr", h_dr), ("h_den_msd", h_den_msd), ("h_num_msd", h_num_msd),
                    ("h_den_dr", h_den_dr), ("h_num_dr", h_num_dr), ("h_den_pt", h_den_pt), ("h_num_pt", h_num_pt)]:
        src = tf.Get(name)
        if src: 
            h.Add(src)

    tf.Close()

h_eff_msd = h_num_msd.Clone("h_eff_msd")
h_eff_msd.Divide(h_den_msd)
h_eff_msd.SetTitle("Efficiency vs m_{softdrop};m_{softdrop} [GeV];Efficiency")

h_eff_pt = h_num_pt.Clone("h_eff_pt")
h_eff_pt.Divide(h_den_pt)
h_eff_pt.SetTitle("Efficiency vs p_{T};p_{T} [GeV];Efficiency")

h_eff_dr = h_num_dr.Clone("h_eff_dr")
h_eff_dr.Divide(h_den_dr)
h_eff_dr.SetTitle("Efficiency vs #DeltaR;#DeltaR;Efficiency")

#====uproot====
out = ROOT.TFile("output_uproot/combined_histograms.root", "RECREATE")

#=====root=====
#out = ROOT.TFile("output/combined_histograms.root", "RECREATE")

for h in [h_msd, h_pt, h_dr, h_den_msd, h_num_msd, h_den_dr, h_num_dr, h_den_pt, h_num_pt,
          h_eff_msd, h_eff_pt, h_eff_dr]:
    h.Write()
out.Close()

In [9]:
def draw(hist, filename, ylog=False, yrange=None, xrange=None, style=True):        
    c = ROOT.TCanvas("c_" + hist.GetName(), hist.GetTitle(), 800, 600)
    if ylog:
        c.SetLogy()
        
    hist.SetLineWidth(1)
    hist.SetLineColor(ROOT.kBlue+1)
    if xrange:
        hist.GetXaxis().SetRangeUser(*xrange)
    if yrange:
        hist.GetYaxis().SetRangeUser(*yrange)
        
    if style:  # draw as histogram
        hist.SetMarkerStyle(1)
        hist.SetMarkerSize(0)
        hist.Draw("HIST E")
    else:      # draw as efficiency points
        hist.SetMarkerStyle(20)
        hist.SetMarkerSize(0.7)
        hist.SetLineWidth(1)
        hist.Draw("E1")

    #=====root=====
    #c.SaveAs(os.path.join("Plots", filename))
    #c.SaveAs(os.path.join("/eos/user/s/scavanau/Mono-V/Genmatching/Plots", filename))

    #====uproot====
    c.SaveAs(os.path.join("Plots_uproot", filename))
    c.SaveAs(os.path.join("/eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot", filename))
    
    c.Close()

draw(h_pt,  "pt.png")
draw(h_msd, "msd.png")
draw(h_dr,  "deltaR.png", xrange=(0, 2.0))
draw(h_eff_msd, "eff_msoftdrop.png", yrange=(0, 1.01), style=False)
draw(h_eff_pt,  "eff_pt.png", yrange=(0, 1.01), style=False)
draw(h_eff_dr,  "eff_deltaR.png", yrange=(0, 1.01), style=False)

Info in <TCanvas::Print>: png file Plots_uproot/pt.png has been created
Info in <TCanvas::Print>: png file /eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot/pt.png has been created
Info in <TCanvas::Print>: png file Plots_uproot/msd.png has been created
Info in <TCanvas::Print>: png file /eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot/msd.png has been created
Info in <TCanvas::Print>: png file Plots_uproot/deltaR.png has been created
Info in <TCanvas::Print>: png file /eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot/deltaR.png has been created
Info in <TCanvas::Print>: png file Plots_uproot/eff_msoftdrop.png has been created
Info in <TCanvas::Print>: png file /eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot/eff_msoftdrop.png has been created
Info in <TCanvas::Print>: png file Plots_uproot/eff_pt.png has been created
Info in <TCanvas::Print>: png file /eos/user/s/scavanau/Mono-V/Genmatching/Plots_uproot/eff_pt.png has been created
Info in <TCanvas::Print>: png file Plots

In [49]:
import ROOT, os, glob
import numpy as np
ifile = ROOT.TFile("5a7f65c2-526f-4fc1-872a-7b9b88423e10.root")
itree = ifile.Get("Events")
branches = itree.GetListOfBranches()
#for branch in branches:
 #   print(branch.GetName())
nEvents = itree.GetEntries()

start = []
with_status62 = []
with_Z = []
with_W = []

print("-,     pdg, mom,     (pt, eta, phi)")
for ev in range(nEvents):
    itree.GetEntry(ev)
    start.append(ev)
    for i in range(itree.nGenPart):
        status = itree.GenPart_status[i]
        pdg = itree.GenPart_pdgId[i]
        mom = itree.GenPart_genPartIdxMother[i]
        eta = itree.GenPart_eta[i]
        mass = itree.GenPart_mass[i]
        phi = itree.GenPart_phi[i]
        pt = itree.GenPart_pt[i]
        print([i], status, pdg, mom,"        (",pt, eta, phi,")")
        if not itree.GenPart_status[i] == 62:
            continue
        with_status62.append(ev)
        
        #print(status, pdg, mom, pt, eta, phi)
        if pdg == 23:
            with_Z.append(ev)
            print("-----------")
            print([i], status, pdg, mom, "        (",pt, eta, phi,")")
            print("-----------")
        elif abs(pdg) == 24:
            with_W.append(ev)
            print([i], status, pdg, mom, "        (",pt, eta, phi,")")
            print("-----------")
    break

#print("nEvents in WZ File:", len(start))
#print(len(with_status62))
#print("nEvents with status 62 pdg Z:", len(with_Z))
#print("nEvents with status 62 pdg W:", len(with_W))

-,     pdg, mom,     (pt, eta, phi)
[0] 21 2 -1         ( 0.0 23424.0 0.0 )
[1] 21 -1 -1         ( 0.0 -22784.0 0.0 )
[2] 22 23 0         ( 19.9375 4.109375 -0.73828125 )
[3] 22 24 0         ( 19.9375 0.4599609375 2.40625 )
[4] 44 23 2         ( 19.625 4.125 -0.744140625 )
[5] 44 24 3         ( 23.4375 0.40625 2.4609375 )
[6] 44 23 4         ( 17.5625 4.234375 -0.78125 )
[7] 44 24 5         ( 23.5625 0.40234375 2.4609375 )
[8] 62 23 6         ( 16.75 4.28125 -0.826171875 )
-----------
[8] 62 23 6         ( 16.75 4.28125 -0.826171875 )
-----------
[9] 62 24 7         ( 22.625 0.4052734375 2.40625 )
[9] 62 24 7         ( 22.625 0.4052734375 2.40625 )
-----------
[10] 23 3 8         ( 8.90625 4.890625 -1.93359375 )
[11] 23 -3 8         ( 15.0625 0.787109375 -0.26953125 )
[12] 52 3 10         ( 8.3125 4.890625 -1.93359375 )
[13] 23 -3 9         ( 40.5 -0.578125 2.3125 )
[14] 23 4 9         ( 18.125 1.390625 -0.955078125 )
[15] 71 4 14         ( 13.96875 1.46484375 -1.078125 )
[16] 71 -3 13